In [76]:
import calendar
import os
import pandas as pd

# from exel_loader import df_operations, df_rigs, df_circ_data
# from db_loader import df_osi_operations, df_meters


# temp read from temp files

DATA_DIR = r'K:/DOP/OED/METHOD&TOOLS/3 - PROJECTS/2 - ON GOING/PYTHON/Scripts/OEE/python/data'

def read_csv(filename):
    path = os.path.join(DATA_DIR, filename)
    return pd.read_csv(path)

df_circ_data = read_csv('df_circ_data.csv')
df_meters = read_csv('df_meters.csv')
df_operations = read_csv('df_operations.csv')
df_osi_operations = read_csv('df_osi_operations.csv')
df_rigs = read_csv('df_rigs.csv')




# Функция для преобразования имён колонок:
def clean_column_names(columns):
    return [col.replace('_', ' ').title() for col in columns]

# Очищаем и переименовываем колонки сразу после чтения
df_circ_data.columns = clean_column_names(df_circ_data.columns)
df_meters.columns = clean_column_names(df_meters.columns)
df_operations.columns = clean_column_names(df_operations.columns)
df_osi_operations.columns = clean_column_names(df_osi_operations.columns)
df_rigs.columns = clean_column_names(df_rigs.columns)



In [77]:
# data modif 

def add_difference_column(df, col1, col2):
    """
    Возвращает Series: разница между двумя колонками с заменой NaN на 0.
    """
    return (df[col1].fillna(0) - df[col2].fillna(0)).clip(lower=0)


def add_ratio_column(df, numerator_col, denominator_col):
    """
    Возвращает Series: отношение numerator_col / denominator_col
    с защитой от деления на 0, заменой NaN на 0 и ограничением [0, 1].
    """
    num = df[numerator_col].fillna(0)
    denom = df[denominator_col].replace(0, pd.NA).fillna(pd.NA)
    ratio = num / denom
    return ratio.fillna(0).clip(0, 1)



# Работа с df_operations
df_operations = df_operations[['Event', 'Category']]
df_operations = df_operations[df_operations['Category'].notna()].reset_index(drop=True)


# Преобразуем Starttime и Endtime в datetime
df_osi_operations['Starttime'] = pd.to_datetime(df_osi_operations['Starttime'], errors='coerce')
df_osi_operations['Endtime'] = pd.to_datetime(df_osi_operations['Endtime'], errors='coerce')

# Добавляем колонку Duration в df_osi_operations
df_osi_operations['Duration'] = df_osi_operations['Endtime'] - df_osi_operations['Starttime']
df_osi_operations['Duration'] = df_osi_operations['Duration'].dt.total_seconds() / 3600

# Очищаем и фильтруем df_rigs
df_rigs = df_rigs[df_rigs['Rig-Acquire'].notna()].reset_index(drop=True)

# Приведение к одному формату для слияния
df_osi_operations['Equipment'] = df_osi_operations['Equipment'].str.strip().str.lower()
df_rigs['Rig-Osidem'] = df_rigs['Rig-Osidem'].str.strip().str.lower()


In [78]:
# Слияние
df_merged = df_osi_operations.merge(
    df_rigs,
    left_on='Equipment',
    right_on='Rig-Osidem',
    how='left'
)

df_temp = df_merged.drop(columns=['Equipment', 'Rig-Osidem'])

# Очистка колонок event
df_temp['Event'] = df_temp['Event'].str.strip().str.lower()
df_operations['Event'] = df_operations['Event'].str.strip().str.lower()

# Слияние с категориями
df_operations_total = df_temp.merge(df_operations, on='Event', how='left')
df_operations_total['Category'] = df_operations_total['Category'].fillna('Standard Work')

# Приведение endtime к datetime и фильтрация
df_operations_total['Endtime'] = pd.to_datetime(df_operations_total['Endtime'])
df_operations_total = df_operations_total.dropna(subset=['Endtime'])

In [79]:
# Добавляем year и month
df_operations_total['Year-Month'] = df_operations_total['Endtime'].dt.strftime('%Y-%m')

# Сводная таблица pivot
pivot_df = df_operations_total.pivot_table(
    index=['Drillcompany', 'Rig-Acquire', 'Year-Month', 'Tipe Of Circulation'],
    columns='Category',
    values='Duration',
    aggfunc='sum',
    fill_value=0
).reset_index()

In [80]:
pivot_df = pivot_df.sort_values(
    by=['Drillcompany', 'Rig-Acquire', 'Year-Month'],
    ascending=[True, True, True]
).reset_index(drop=True)

# Функция для подсчёта часов в месяце
def hours_in_month(row):
    year_month_str = row['Year-Month']  # предполагается, что у вас есть колонка с таким именем
    year, month = map(int, year_month_str.split('-'))
    days = calendar.monthrange(year, month)[1]
    return days * 24

In [81]:
pivot_df['H In Month'] = pivot_df.apply(hours_in_month, axis=1)

In [82]:
# Преобразование ENDDATE в datetime
df_meters['Enddate'] = pd.to_datetime(df_meters['Enddate'], errors='coerce')
df_meters.dropna(subset=['Enddate'], inplace=True)
df_meters['Year-Month'] = df_meters['Enddate'].dt.strftime('%Y-%m')


In [83]:
pivot2 = pd.pivot_table(
    df_meters,
    index=['Year-Month', 'Drillrig'],
    columns='Holestatus',
    values='Holeid',
    aggfunc='count',
    fill_value=0
)

In [84]:
pivot2

Holestatus                 ACCEPTED  LIQUID  NOT PROFITABLE
Year-Month Drillrig                                        
2020-01    DBKAZ40_1-KAT          7       0               0
           PRAKLA_04-BGP          2       2               0
           TWS685-1-KAT           3       0               0
           TWS685-2-KAT           1       0               0
           TWS685-3-KAT           2       0               0
...                             ...     ...             ...
2025-06    ZIF1200_28-BGP         1       0               0
           ZIF1200_29-BGP         2       0               0
           ZMO1500-4-KAT          1       0               0
           ZMO1500-5-KAT          1       0               0
           ZMO1500-6-KAT          1       0               0

[2550 rows x 3 columns]

In [85]:
pivot_df

Category,Drillcompany,Rig-Acquire,Year-Month,Tipe Of Circulation,Planned_downtime,Standard Work,Unplanned_downtime_losses,H In Month
0,BurGeoProekt,ZIF1200_01-BGP,2020-02,Direct,0.0,0.0,276.0,696
1,BurGeoProekt,ZIF1200_01-BGP,2020-03,Direct,47.0,420.0,278.0,744
2,BurGeoProekt,ZIF1200_01-BGP,2020-04,Direct,17.0,182.0,28.0,720
3,BurGeoProekt,ZIF1200_01-BGP,2021-01,Direct,9.0,7015.0,93.0,744
4,BurGeoProekt,ZIF1200_01-BGP,2021-02,Direct,84.0,397.0,191.0,672
...,...,...,...,...,...,...,...,...
2812,TechnoService-Eng,ZIF1200_16-TSE,2023-03,Direct,89.0,509.0,90.0,744
2813,TechnoService-Eng,ZIF1200_16-TSE,2023-04,Direct,76.0,468.0,106.0,720
2814,TechnoService-Eng,ZIF1200_16-TSE,2023-05,Direct,76.0,486.0,119.0,744
2815,TechnoService-Eng,ZIF1200_16-TSE,2023-06,Direct,77.0,405.0,133.0,720


In [86]:
depth_sum = df_meters.groupby(['Year-Month', 'Drillrig'])['Depth'].sum()
pivot2['Depth'] = depth_sum
meters_pivot = pivot2.reset_index()

# Приведение к верхнему регистру и очистка пробелов
pivot_df['Rig-Acquire'] = pivot_df['Rig-Acquire'].str.strip().str.upper()
meters_pivot['Drillrig'] = meters_pivot['Drillrig'].str.strip().str.upper()

total_merged_df = pd.merge(
    pivot_df,
    meters_pivot,
    left_on=['Rig-Acquire', 'Year-Month'],
    right_on=['Drillrig', 'Year-Month'],
    how='left'
)

In [87]:
total_merged_df.drop(columns='Rig-Acquire', inplace=True)

In [88]:
# Расчёты Planned Production Time и Planned Factor с ограничениями
total_merged_df['Planned Production Time'] = add_difference_column(total_merged_df, 'H In Month', 'Planned_downtime')

In [89]:
total_merged_df['Planned Factor'] = add_ratio_column(
    total_merged_df, 'Planned Production Time', 'H In Month'
)

# Gross Operating Time (GOT) и Availability
total_merged_df['Gross Operating Time'] = add_difference_column(
    total_merged_df, 'Planned Production Time', 'Unplanned_downtime_losses'
)

In [90]:
total_merged_df['Availability'] = add_ratio_column(
    total_merged_df, 'Gross Operating Time', 'Planned Production Time'
)

# Назначаем коэффициенты по типу циркуляции
circ_avg_drilling = df_circ_data.set_index('Circ')['Standard Avarage Drilling, M/H'].to_dict()
time_to_well_drill = df_circ_data.set_index('Circ')['Time To Well Drill, H'].to_dict()

# Записываем два столбца в df
total_merged_df['Circulation Coeff'] = total_merged_df['Tipe Of Circulation'].map(circ_avg_drilling)
total_merged_df['Well Drill Coef'] = total_merged_df['Tipe Of Circulation'].map(time_to_well_drill)


# Потенциальная глубина бурения
total_merged_df['Potential Depth'] = (
    total_merged_df['Gross Operating Time'] * total_merged_df['Circulation Coeff']
)

In [91]:
total_merged_df['Net Operating Time'] = total_merged_df['Depth'].div(
    total_merged_df['Circulation Coeff']
).fillna(0)

In [92]:
total_merged_df['Speed Losses'] = add_difference_column(
    total_merged_df, 'Gross Operating Time', 'Net Operating Time'
)

In [93]:
total_merged_df

,Drillcompany,Year-Month,Tipe Of Circulation,Planned_downtime,Standard Work,Unplanned_downtime_losses,H In Month,Drillrig,ACCEPTED,LIQUID,...,Depth,Planned Production Time,Planned Factor,Gross Operating Time,Availability,Circulation Coeff,Well Drill Coef,Potential Depth,Net Operating Time,Speed Losses
0,BurGeoProekt,2020-02,Direct,0.0,0.0,276.0,696,ZIF1200_01-BGP,0.0,1.0,...,80.0,696.0,1.000000,420.0,0.603448,2.7,210,1134.0,29.629630,390.370370
1,BurGeoProekt,2020-03,Direct,47.0,420.0,278.0,744,ZIF1200_01-BGP,2.0,0.0,...,698.0,697.0,0.936828,419.0,0.601148,2.7,210,1131.3,258.518519,160.481481
2,BurGeoProekt,2020-04,Direct,17.0,182.0,28.0,720,ZIF1200_01-BGP,2.0,0.0,...,697.0,703.0,0.976389,675.0,0.960171,2.7,210,1822.5,258.148148,416.851852
3,BurGeoProekt,2021-01,Direct,9.0,7015.0,93.0,744,NaN,NaN,NaN,...,NaN,735.0,0.987903,642.0,0.873469,2.7,210,1733.4,0.000000,642.000000
4,BurGeoProekt,2021-02,Direct,84.0,397.0,191.0,672,ZIF1200_01-BGP,2.0,0.0,...,998.0,588.0,0.875000,397.0,0.675170,2.7,210,1071.9,369.629630,27.370370
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2812,TechnoService-Eng,2023-03,Direct,89.0,509.0,90.0,744,ZIF1200_16-TSE,2.0,0.0,...,587.0,655.0,0.880376,565.0,0.862595,2.7,210,1525.5,217.407407,347.592593
2813,TechnoService-Eng,2023-04,Direct,76.0,468.0,106.0,720,ZIF1200_16-TSE,4.0,0.0,...,1132.0,644.0,0.894444,538.0,0.835404,2.7,210,1452.6,419.259259,118.740741
2814,TechnoService-Eng,2023-05,Direct,76.0,486.0,119.0,744,ZIF1200_16-TSE,2.0,0.0,...,565.0,668.0,0.897849,549.0,0.821856,2.7,210,1482.3,209.259259,339.740741
2815,TechnoService-Eng,2023-06,Direct,77.0,405.0,133.0,720,ZIF1200_16-TSE,3.0,0.0,...,1043.0,643.0,0.893056,510.0,0.793157,2.7,210,1377.0,386.296296,123.703704


In [94]:
total_merged_df['Performance'] = add_ratio_column(total_merged_df, 'Net Operating Time', 'Gross Operating Time')

total_merged_df['Quality Losses'] =  total_merged_df.get('Well Drill Coef', 0) * total_merged_df.get('LIQUID', 0)

# Valuable Operating Time
total_merged_df['Valuable Operating Time'] = add_difference_column(
    total_merged_df, 'Net Operating Time', 'Quality Losses'
    )


C:\Users\ykarabekov\AppData\Local\Temp\ipykernel_1880\1726980654.py:18: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  return ratio.fillna(0).clip(0, 1)


In [95]:
total_merged_df['Quality Losses'].describe()

count    2433.000000
mean       39.207152
std        90.803698
min         0.000000
25%         0.000000
50%         0.000000
75%         0.000000
max       630.000000
Name: Quality Losses, dtype: float64

In [96]:
total_merged_df['Quality'] = add_ratio_column(
    total_merged_df, 'Valuable Operating Time', 'Net Operating Time'
    )

C:\Users\ykarabekov\AppData\Local\Temp\ipykernel_1880\1726980654.py:18: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  return ratio.fillna(0).clip(0, 1)


In [97]:
total_merged_df.columns

Index(['Drillcompany', 'Year-Month', 'Tipe Of Circulation', 'Planned_downtime',
       'Standard Work', 'Unplanned_downtime_losses', 'H In Month', 'Drillrig',
       'ACCEPTED', 'LIQUID', 'NOT PROFITABLE', 'Depth',
       'Planned Production Time', 'Planned Factor', 'Gross Operating Time',
       'Availability', 'Circulation Coeff', 'Well Drill Coef',
       'Potential Depth', 'Net Operating Time', 'Speed Losses', 'Performance',
       'Quality Losses', 'Valuable Operating Time', 'Quality'],
      dtype='object')

In [98]:
total_merged_df['OEE'] = (
    total_merged_df['Availability'] *
    total_merged_df['Performance'] *
    total_merged_df['Quality']
)


total_merged_df['TRS'] = total_merged_df['OEE'] * total_merged_df['Planned Factor']

In [99]:
total_merged_df.columns = clean_column_names(total_merged_df.columns)

In [100]:
total_merged_df = total_merged_df.rename(columns={'Drillrig': 'Rig'})

In [101]:
df_losses = pd.melt(
    total_merged_df,
    id_vars=[
        'Drillcompany',
        'Rig',
        'Year-Month',
        'Tipe Of Circulation'
    ],
    value_vars=[
        'Planned Downtime',
        'Unplanned Downtime Losses',
        'Speed Losses',
        'Quality Losses'
    ],
    var_name='Loss Type',
    value_name='Loss Value'
)

In [102]:
df_productivity = pd.melt(
    total_merged_df,
    id_vars=[
        'Drillcompany',
        'Rig',
        'Year-Month',
        'Tipe Of Circulation'
    ],
    value_vars=[
        'Planned Factor',
        'Availability',
        'Performance',
        'Quality',
        'Oee',
        'Trs'
    ],
    var_name='Productivity Type',
    value_name='Productivity Value'
)

In [103]:
df_operations_total['Category'] = df_operations_total['Category'].replace({
    'Planned_downtime': 'Planned Downtime',
    'Unplanned_downtime_losses': 'Unplanned Downtime Losses'
})

In [104]:
df_operations_total = df_operations_total.rename(columns={'Rig-Acquire': 'Rig'})

In [105]:
df_operations_total['Event Category'] = df_operations_total['Event'].str.split('_').str[0].str.upper()
df_operations_total['Event'] = df_operations_total['Event'].str.split('_', n=1).str[1].str.capitalize()

In [106]:
df_events_duration = df_operations_total[
    [
        'Year-Month',
        "Category",
        'Drillcompany',
        'Rig',
        'Tipe Of Circulation',
        "Event Category",
        "Event",
        "Duration"
    ]
].sort_values(
    by=['Year-Month', 'Rig', 'Category', 'Duration'],
    ascending=[True, True, True, False],
)



In [107]:
df_meters.dropna(subset=['Drillcompany', 'Drillrig'], inplace=True)

In [108]:
df_meters.columns = [
    'Hole ID',
    'Drilling Company',
    'Rig',
    'Purpose',
    'Status',
    'End_Date',
    'Depth_m',
    'Year-Month'
]

df_meters = df_meters.drop(columns=['End_Date'])



In [109]:
print(df_meters)
print(df_events_duration)
print(df_productivity)
print(df_losses)


             Hole ID   Drilling Company             Rig  Purpose    Status  \
0     MSK39_08_09_A4       BurGeoProekt   PRAKLA_01-BGP  PRODUCT  ACCEPTED   
1     MSK03_06_02_A_       BurGeoProekt   PRAKLA_03-BGP  PRODUCT    LIQUID   
2     MSK03_04_05_A_       BurGeoProekt   PRAKLA_04-BGP  PRODUCT    LIQUID   
3     MSK03_04_09_A_       BurGeoProekt   PRAKLA_04-BGP  PRODUCT    LIQUID   
4     MSK04_05_04_A_       BurGeoProekt  ZIF1200_29-BGP   INJECT  ACCEPTED   
...              ...                ...             ...      ...       ...   
7096  TNU39_08_05_A_  TechnoService-Eng  ZIF1200_01-TSE  PRODUCT  ACCEPTED   
7097  TNU39_08_03_B_  TechnoService-Eng  ZIF1200_08-TSE  PRODUCT  ACCEPTED   
7098  TNU39_08_07_A_  TechnoService-Eng  ZIF1200_04-TSE  PRODUCT  ACCEPTED   
7099  TNU39_13_09_A_  TechnoService-Eng  ZIF1200_03-TSE   INJECT  ACCEPTED   
7100  TNU39_13_10_A_  TechnoService-Eng  ZIF1200_08-TSE   INJECT  ACCEPTED   

      Depth_m Year-Month  
0       508.0    2020-04  
1       4

In [110]:
unique_dates_series = pd.Series(
    sorted(df_productivity['Year-Month'].dropna().unique()),
    name='Year-Month'
)

unique_dates_series


0     2020-02
1     2020-03
2     2020-04
3     2020-08
4     2020-09
       ...   
57    2025-02
58    2025-03
59    2025-04
60    2025-05
61    2025-06
Name: Year-Month, Length: 62, dtype: object

In [111]:
df_productivity['Productivity Type'].value_counts()

df_losses['Loss Type']

0        Planned Downtime
1        Planned Downtime
2        Planned Downtime
3        Planned Downtime
4        Planned Downtime
               ...       
11263      Quality Losses
11264      Quality Losses
11265      Quality Losses
11266      Quality Losses
11267      Quality Losses
Name: Loss Type, Length: 11268, dtype: object